In [1]:
!pip install git+https://github.com/philjd/contiguous_pytorch_params.git
!pip list | grep torch

  Cloning https://github.com/philjd/contiguous_pytorch_params.git to /tmp/pip-req-build-lrn4q0hr
  Running command git clone -q https://github.com/philjd/contiguous_pytorch_params.git /tmp/pip-req-build-lrn4q0hr
  Created wheel for contiguous-params: filename=contiguous_params-1.0-cp36-none-any.whl size=5164 sha256=8604134ed9a9678dbe6651db516da95fea68887f106bb57d74b27e44541b3d55
  Stored in directory: /tmp/pip-ephem-wheel-cache-368bdglm/wheels/63/df/a3/834ec4f3ca06e3ce88ac71c1089075a15988d06f6e820b8717
Successfully built contiguous-params
torch                         1.6.0+cu101    
torchsummary                  1.5.1          
torchtext                     0.3.1          
torchvision                   0.7.0+cu101    


In [16]:
import os
import random
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils import spectral_norm
from torch.cuda.amp import GradScaler, autocast
from torch.utils.data import DataLoader
from torchvision import datasets as dsets
from torchvision import transforms as T
from torchsummary import summary

from contiguous_params import ContiguousParams

In [9]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.random.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True


set_seed(1234)

In [10]:
cifar10 = dsets.CIFAR10(
    "../data",
    train=True,
    download=True,
    transform=T.Compose([
        T.Resize((32, 32)),
        T.ToTensor(),
        T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
    ])
)

Files already downloaded and verified


In [11]:
loader = DataLoader(
    cifar10,
    batch_size=64,
    shuffle=True,
    num_workers=4,
    pin_memory=True,
)

In [12]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

In [29]:
class DCGAN(nn.Module):

    def __init__(self, nz, ngf, nc):
        super().__init__()
        self.nz = nz
        self.ngf = ngf
        self.nc = nc

        # [B, nz, 1, 1] --> [B, ngf*8, 4, 4]
        self.layer1 = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, kernel_size=4, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(inplace=True)
        )

        # [B, ngf*8, 4, 4] --> [B, ngf*4, 8, 8]
        self.layer2 = nn.Sequential(
            nn.ConvTranspose2d(ngf * 8, ngf * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(inplace=True)
        )

        # [B, ngf*4, 8, 8] --> [B, ngf*2, 16, 16]
        self.layer3 = nn.Sequential(
            nn.ConvTranspose2d(ngf * 4, ngf * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(inplace=True)
        )

        # [B, ngf*2, 16, 16] --> [B, ngf, 32, 32]
        self.layer4 = nn.Sequential(
            nn.ConvTranspose2d(ngf *2, ngf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(inplace=True)
        )

        self.logits = nn.Sequential(
            nn.Conv2d(ngf, nc, kernel_size=1, stride=1, padding=0),
            nn.Tanh()
        )

    def forward(self, x):
        """forward propagation

        :param x: (Tensor) the input latent code of its shape [B, nz]
        """

        x = x.view(-1, self.nz, 1, 1)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        logits = self.logits(x)

        return x

In [30]:
summary(DCGAN(100, 64, 3).to("cuda:0"), (100,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
   ConvTranspose2d-1            [-1, 512, 4, 4]         819,200
       BatchNorm2d-2            [-1, 512, 4, 4]           1,024
              ReLU-3            [-1, 512, 4, 4]               0
   ConvTranspose2d-4            [-1, 256, 8, 8]       2,097,152
       BatchNorm2d-5            [-1, 256, 8, 8]             512
              ReLU-6            [-1, 256, 8, 8]               0
   ConvTranspose2d-7          [-1, 128, 16, 16]         524,288
       BatchNorm2d-8          [-1, 128, 16, 16]             256
              ReLU-9          [-1, 128, 16, 16]               0
  ConvTranspose2d-10           [-1, 64, 32, 32]         131,072
      BatchNorm2d-11           [-1, 64, 32, 32]             128
             ReLU-12           [-1, 64, 32, 32]               0
           Conv2d-13            [-1, 3, 32, 32]             195
             Tanh-14            [-1, 3,

In [25]:
class Discriminator(nn.Module):

    def __init__(self, ndf):
        super().__init__()
        self.ndf = ndf

        self.layer1 = nn.Sequential(
            spectral_norm(nn.Conv2d(nc, ndf, kernel_size=4, stride=2, padding=11, bias=False)),
            nn.BatchNorm2d(ndf),
            nn.LeakyReLU(inplace=True)
        )

        

torch.Size([100])